In [7]:
import pandas as pd
import numpy as np
from datetime import datetime

In [8]:
TotalBudget=100000
# df=pd.read_csv('ASP-SME_KKP-SM-TRade.csv',index_col='Date/Time',parse_dates=['Date/Time'],dayfirst=True)
df=pd.read_csv('TestPort.csv',parse_dates=['Date/Time'],dayfirst=True)
# df.index.set_names('date',inplace=True)
df.columns=['symbol','date','price']

list_symbols=df['symbol'].unique()
print(list_symbols)
df.info()
print(df.tail())

['KKP-SM-CAP' 'ASP-SME' 'TISCOFLEXP']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2181 entries, 0 to 2180
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   symbol  2181 non-null   object        
 1   date    2181 non-null   datetime64[ns]
 2   price   2181 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 51.2+ KB
          symbol       date  price
2176  TISCOFLEXP 2022-12-30  61.68
2177  TISCOFLEXP 2023-01-03  61.76
2178  TISCOFLEXP 2023-01-04  60.98
2179  TISCOFLEXP 2023-01-05  60.75
2180  TISCOFLEXP 2023-01-06  60.56


In [9]:
dfPlan=pd.read_excel('ListTradeTran.xlsx',engine="openpyxl",sheet_name='TradePlan', \
                     parse_dates=['trade_date'],dtype ={'portion_pct': np.float64})
dfAssetInfo=pd.read_excel('ListTradeTran.xlsx',engine="openpyxl",sheet_name='AssetInfo',\
                    index_col='symbol', dtype ={'buy_comm_pct': np.float64,'sell_comm_pct': np.float64})

tradeDateList= [ d for d in list(dfPlan['trade_date']) ]

print(tradeDateList)
print("================================================================================================")

print(dfPlan)
print(dfAssetInfo)

# verify b and sell , sum of each action have to be 100  

[Timestamp('2020-12-01 00:00:00'), Timestamp('2021-01-05 00:00:00'), Timestamp('2021-01-15 00:00:00'), Timestamp('2021-02-01 00:00:00'), Timestamp('2022-01-19 00:00:00'), Timestamp('2022-02-10 00:00:00'), Timestamp('2022-03-03 00:00:00'), Timestamp('2022-04-29 00:00:00')]
  trade_date  portion_pct action
0 2020-12-01         30.0      b
1 2021-01-05         20.0      b
2 2021-01-15         20.0      b
3 2021-02-01         30.0      b
4 2022-01-19         40.0      s
5 2022-02-10         20.0      s
6 2022-03-03         20.0      s
7 2022-04-29         20.0      s
            buy_comm_pct  sell_comm_pct
symbol                                 
KKP-SM-CAP           1.0            0.0


In [10]:
def set_commision_by_action(item):
  x=item['symbol']  
  if item['action']=='b':
    comm_type='buy_comm_pct'
  elif  item['action']=='s':
    comm_type='sell_comm_pct'
  try:
     return dfAssetInfo.loc[x,comm_type]
  except Exception as ex:
     print(f"Error : no {x} to get commission")
     return 0
def get_qty_buy(item):
    xCost=(item['portion_pct']/100)*TotalBudget
    xPrice=item['price_on_comm']
    qty=xCost/xPrice
    return qty
def get_qty_sell(item,TotalQty):
    qty=(item['portion_pct']/100)*TotalQty
    return qty

In [25]:
dict_dfTran={}
dfSummary=pd.DataFrame(columns=['Symbol','BuyValue','AvgBuyPrice','SellValue','AvgSellPrice','PL','Percent_PL'])
for index,item in dfAssetInfo.iterrows():
    symbol=index
    print(symbol)
    dfBS=df[ (df['symbol']==symbol) & (df['date'].isin(tradeDateList))]
    dfBS=dfBS.merge(dfPlan,how='inner',left_on='date',right_on='trade_date')
    
    dfBS['comm']=dfBS.apply(set_commision_by_action,axis=1)
    dfBS['price_on_comm']=dfBS.apply(lambda item : item['price']*(1+ (item['comm']/100)) if item['action']=='b' else item['price']*(1-(item['comm']/100)), axis=1)
    dfBS['price_on_comm']=dfBS['price_on_comm'].round(2)
    
    dfBS=dfBS.drop(columns=['trade_date'])
    #print(dfBS)
    
    print("================Buy=================")
    buyDF=dfBS.query("action=='b'")    
    # cal qury and actual value and  total_val and totl qury
    buyDF['qty']=buyDF.apply(get_qty_buy,axis=1)
    buyDF['value']=buyDF['qty']*buyDF['price_on_comm']
    buyDF['action_value']=buyDF['value']
    print(buyDF[['date' ,'price_on_comm','qty','value','portion_pct']])
    
    TotalQty=buyDF['qty'].sum()
    TotalValue=round(buyDF['value'].sum(),2)
    AvgPrice=round(TotalValue/TotalQty,2)
    # print(f"Total Buy {symbol} : qty={TotalQty} and value={TotalValue} at AvgPrice={AvgPrice}")
    
    print("=================Sell=================")
    sellDF=dfBS.query("action=='s'")  
    sellDF['qty']=sellDF.apply(get_qty_sell,axis=1,args=(TotalQty,))
    sellDF['value']=sellDF['qty']*sellDF['price_on_comm']
    sellDF['action_value']=sellDF['value']*-1
    print(sellDF[['date' ,'price_on_comm','qty','value','portion_pct']])
    
    SellTotalQty=sellDF['qty'].sum()
    SellTotalValue=round(sellDF['value'].sum(),2)
    SellAvgPrice=round(SellTotalValue/SellTotalQty,2)
    
    # print(f"Total Sell {symbol} : qty={SellTotalQty} and value={SellTotalValue} at AvgPrice={SellAvgPrice}")
    
    PL=round(SellTotalValue-TotalValue,0)
    PCT_PL=round(PL/TotalValue*100,2)
    
    summary_data={'Symbol':symbol,'BuyValue':TotalValue,'AvgBuyPrice':AvgPrice,'SellValue':SellTotalValue,'AvgSellPrice':SellAvgPrice, \
                  'PL':PL,'Percent_PL':PCT_PL
                 }
    dfSummary=dfSummary.append(summary_data,ignore_index=True)
    
    dfTran=pd.concat([buyDF,sellDF],axis=0)
    dfTran[['value','action_value']]=dfTran[['value','action_value']].round(0)
    
    dict_dfTran[symbol]=dfTran
    
    
    
    

    


KKP-SM-CAP
================Buy=================
        date  price_on_comm          qty    value  portion_pct
0 2020-12-01          12.94  2318.392581  30000.0         30.0
1 2021-01-05          13.58  1472.754050  20000.0         20.0
2 2021-01-15          13.84  1445.086705  20000.0         20.0
3 2021-02-01          14.01  2141.327623  30000.0         30.0
=================Sell=================
        date  price_on_comm          qty         value  portion_pct
4 2022-01-19          18.51  2951.024384  54623.461344         40.0
5 2022-02-10          18.06  1475.512192  26647.750186         20.0
6 2022-03-03          18.21  1475.512192  26869.077015         20.0
7 2022-04-29          18.61  1475.512192  27459.281891         20.0


C:\Users\pongthsa\AppData\Local\Temp\ipykernel_29440\2972983705.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buyDF['qty']=buyDF.apply(get_qty_buy,axis=1)
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_29440\2972983705.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buyDF['value']=buyDF['qty']*buyDF['price_on_comm']
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_29440\2972983705.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [26]:
print(dfSummary)

,Symbol,BuyValue,AvgBuyPrice,SellValue,AvgSellPrice,PL,Percent_PL
0,KKP-SM-CAP,100000.0,13.55,135599.57,18.38,35600.0,35.6


In [28]:
for symbol,tranDF in dict_dfTran.items():
 print(tranDF)

       symbol       date  price  portion_pct action  comm  price_on_comm  \
0  KKP-SM-CAP 2020-12-01  12.81         30.0      b   1.0          12.94   
1  KKP-SM-CAP 2021-01-05  13.45         20.0      b   1.0          13.58   
2  KKP-SM-CAP 2021-01-15  13.70         20.0      b   1.0          13.84   
3  KKP-SM-CAP 2021-02-01  13.87         30.0      b   1.0          14.01   
4  KKP-SM-CAP 2022-01-19  18.51         40.0      s   0.0          18.51   
5  KKP-SM-CAP 2022-02-10  18.06         20.0      s   0.0          18.06   
6  KKP-SM-CAP 2022-03-03  18.21         20.0      s   0.0          18.21   
7  KKP-SM-CAP 2022-04-29  18.61         20.0      s   0.0          18.61   

           qty     value  action_value  
0  2318.392581  30000.00      30000.00  
1  1472.754050  20000.00      20000.00  
2  1445.086705  20000.00      20000.00  
3  2141.327623  30000.00      30000.00  
4  2951.024384  54623.46     -54623.46  
5  1475.512192  26647.75     -26647.75  
6  1475.512192  26869.08    